In [1]:
#imports 

import numpy as np
import os, shutil

import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import image_dataset_from_directory 
from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split, cross_val_score

import time
import scipy
from scipy import ndimage
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from PIL import Image
import matplotlib as plt

## Image Load In

In [2]:
!ls

README.md
Yuhkai_notebook_scratch.ipynb
data
data.zip
notebooks_scratch


In [3]:
!cd ..

In [4]:
!ls

README.md
Yuhkai_notebook_scratch.ipynb
data
data.zip
notebooks_scratch


In [5]:
!ls

README.md
Yuhkai_notebook_scratch.ipynb
data
data.zip
notebooks_scratch


In [6]:
# load in the train data

#starting from main project directory

train_normal_dir = "./data/chest_xray/train/NORMAL"
train_pneumonia_dir = "./data/chest_xray/train/PNEUMONIA"

imgs_train_normal = [file for file in os.listdir(train_normal_dir) if file.endswith('.jpeg')]
imgs_train_pneumonia = [file for file in os.listdir(train_pneumonia_dir) if file.endswith('.jpeg')]

In [7]:
test_normal_dir = "./data/chest_xray/test/NORMAL"
test_pneumonia_dir = "./data/chest_xray/test/PNEUMONIA"

imgs_test_normal = [file for file in os.listdir(test_normal_dir) if file.endswith('.jpeg')]
imgs_test_pneumonia = [file for file in os.listdir(test_pneumonia_dir) if file.endswith('.jpeg')]

In [8]:
len(imgs_train_normal)

1341

In [9]:
len(imgs_train_pneumonia)

3875

In [10]:
imgs_train_normal[0]

'IM-0115-0001.jpeg'

In [11]:
imgs_train_pneumonia[0]

'person1000_bacteria_2931.jpeg'

In [12]:
len(imgs_test_normal)

234

In [13]:
len(imgs_test_pneumonia)

390

In [17]:
num_train = len(imgs_train_normal) + len(imgs_train_pneumonia)
num_test = len(imgs_test_normal) + len(imgs_test_pneumonia)
num_train, num_test

(5216, 624)

In [16]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [18]:
#rescale images to 64 by 64
#create validation set as 20% of train set

train_folder = "./data/chest_xray/train"
train_gen = train_datagen.flow_from_directory(
        train_folder, 
        target_size=(64, 64), 
        color_mode='grayscale', 
        class_mode='binary',
        subset='training',
        batch_size= num_train)

validation_gen = train_datagen.flow_from_directory(
        train_folder, 
        target_size=(64, 64), 
        color_mode='grayscale', 
        class_mode='binary',
        subset='validation',
        batch_size= num_train)

test_folder = "./data/chest_xray/test"
test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_folder, 
        target_size=(64, 64), color_mode='grayscale', 
        class_mode='binary', 
        batch_size= num_test)

Found 4173 images belonging to 2 classes.
Found 1043 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [19]:
train_images, train_labels = next(train_gen)
val_images, val_labels = next(validation_gen)
test_images, test_labels = next(test_gen)

In [20]:
train_images[0]

array([[[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       ...,

       [[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.00392157],
        [0.00784314],
        [0.00392157]],

       [[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]]], dtype=float32)

In [21]:
val_images[0]

array([[[0.04705883],
        [0.21568629],
        [0.43137258],
        ...,
        [0.30980393],
        [0.13725491],
        [0.03921569]],

       [[0.03921569],
        [0.21568629],
        [0.43137258],
        ...,
        [0.27058825],
        [0.1137255 ],
        [0.02745098]],

       [[0.03137255],
        [0.16862746],
        [0.41960788],
        ...,
        [0.227451  ],
        [0.07450981],
        [0.        ]],

       ...,

       [[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [1.        ],
        [0.00784314],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]]], dtype=float32)

In [22]:
train_images.shape

(4173, 64, 64, 1)

In [23]:
train_images[0].shape

(64, 64, 1)

In [24]:
train_labels[0]

1.0

In [25]:
train_labels.shape

(4173,)

In [26]:
val_images.shape

(1043, 64, 64, 1)

In [27]:
val_labels.shape

(1043,)

In [31]:
#reshaping features for dense layers
train_reshape = train_images.reshape(train_images.shape[0], -1)
val_reshape = val_images.reshape(val_images.shape[0], -1)
test_reshape = test_images.reshape(test_images.shape[0], -1)

print(train_reshape.shape)
print(val_reshape.shape)
print(test_reshape.shape)

(4173, 4096)
(1043, 4096)
(468, 4096)


In [32]:
train_labels.shape

(4173,)

In [33]:
val_labels.shape

(1043,)

In [34]:
test_labels.shape

(468,)

In [38]:
#reshaping labels
train_y = np.reshape(train_labels, (4173,1))
val_y = np.reshape(val_labels, (1043,1))
test_y = np.reshape(test_labels, (468,1))

print(train_y.shape)
print(val_y.shape)
print(test_y.shape)

(4173, 1)
(1043, 1)
(468, 1)


## Baseline Model

In [61]:
baseline_model = models.Sequential()

In [62]:
baseline_model.add(layers.Dense(20, activation='relu', input_shape=(4096,)))
baseline_model.add(layers.Dense(7, activation='relu'))
baseline_model.add(layers.Dense(5, activation='relu'))
baseline_model.add(layers.Dense(1, activation='sigmoid'))

In [63]:
baseline_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 20)                81940     
_________________________________________________________________
dense_13 (Dense)             (None, 7)                 147       
_________________________________________________________________
dense_14 (Dense)             (None, 5)                 40        
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 6         
Total params: 82,133
Trainable params: 82,133
Non-trainable params: 0
_________________________________________________________________


In [65]:
baseline_model.compile(optimizer='sgd', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

In [58]:
train_reshape[0].shape

(4096,)

In [59]:
val_reshape[0].shape

(4096,)

In [66]:
baseline_model.fit(train_reshape, 
                   train_y, 
                   epochs=50, 
                   batch_size=32, 
                   validation_data=(val_reshape, val_y))

Epoch 1/50
131/131 [==============================] - 3s 22ms/step - loss: 0.5403 - accuracy: 0.7402 - val_loss: 0.5084 - val_accuracy: 0.7430
Epoch 2/50
131/131 [==============================] - 0s 1ms/step - loss: 0.4148 - accuracy: 0.7836 - val_loss: 0.3447 - val_accuracy: 0.8849
Epoch 3/50
131/131 [==============================] - 0s 1ms/step - loss: 0.2826 - accuracy: 0.8965 - val_loss: 0.2834 - val_accuracy: 0.8811
Epoch 4/50
131/131 [==============================] - 0s 1ms/step - loss: 0.2603 - accuracy: 0.8967 - val_loss: 0.2325 - val_accuracy: 0.9012
Epoch 5/50
131/131 [==============================] - 0s 1ms/step - loss: 0.2393 - accuracy: 0.9029 - val_loss: 0.2075 - val_accuracy: 0.9099
Epoch 6/50
131/131 [==============================] - 0s 1ms/step - loss: 0.1839 - accuracy: 0.9267 - val_loss: 0.2025 - val_accuracy: 0.9156
Epoch 7/50
131/131 [==============================] - 0s 1ms/step - loss: 0.1804 - accuracy: 0.9315 - val_loss: 0.1682 - val_accuracy: 0.9281
Epoch

In [68]:
results_train = baseline_model.evaluate(train_reshape, train_y)
results_train

131/131 [==============================] - 1s 4ms/step - loss: 0.7116 - accuracy: 0.7501


[0.711613118648529, 0.7500599026679993]

In [70]:
results_val = baseline_model.evaluate(val_reshape, val_y)
results_val

33/33 [==============================] - 0s 11ms/step - loss: 0.9528 - accuracy: 0.7066


[0.9528127908706665, 0.7066155076026917]